In [ ]:
EV Veh

In [1]:
import numpy as np
import pandas as pd

In [9]:
! ls data

ny_ev_registrations_public.xlsx


In [11]:
df = pd.read_excel('data/ny_ev_registrations_public.xlsx', sheet_name = None)